# Implementing a warm-up period

We will implement warm-up as a single event that resets all of our results collection variables.  

This is a simpler approach than including lots of if statements in `simpy` processes.

## 1. Imports 

In [1]:
import numpy as np
import itertools
import simpy

In [25]:
# to reduce code these classes can be found in distribution.py
from distributions import (
    Exponential, 
    Lognormal
)

## 2. Constants

In [3]:
# default mean inter-arrival times(exp)
# time is in days
IAT_STROKES = 1.0

# resources
N_ACUTE_BEDS = 9

# Acute LoS (Lognormal)
ACUTE_LOS_MEAN = 7.0
ACUTE_LOC_STD = 1.0

# sampling settings
N_STREAMS = 2
DEFAULT_RND_SET = 0

# Boolean switch to simulation results as the model runs
TRACE = False

# run variables (units = days)
WU_PERIOD = 0.0
RC_PERIOD = 100

## 2. Helper classes and functions

In [4]:
def trace(msg):
    """
    Turing printing of events on and off.

    Params:
    -------
    msg: str
        string to print to screen.
    """
    if TRACE:
        print(msg)

## 3. Experiment class

In [5]:
class Experiment:
    """
    Encapsulates the concept of an experiment 🧪 for the stroke pathway
    bed blocking simulator. Manages parameters, PRNG streams and results.
    """
    def __init__(
        self,
        random_number_set=DEFAULT_RND_SET,
        n_streams=N_STREAMS,
        iat_strokes=IAT_STROKES,
        acute_los_mean=ACUTE_LOS_MEAN,
        acute_los_std=ACUTE_LOC_STD,
        n_acute_beds=N_ACUTE_BEDS, 
    ):
        """
        The init method sets up our defaults.
        """
        # sampling
        self.random_number_set = random_number_set
        self.n_streams = n_streams

        # store parameters for the run of the model
        self.iat_strokes = iat_strokes
        self.acute_los_mean = acute_los_mean
        self.acute_los_std = acute_los_std

        #  place holder for resources
        self.acute_ward = None
        self.n_acute_beds = n_acute_beds
        
        # initialise results to zero
        self.init_results_variables()

        # initialise sampling objects
        self.init_sampling()

    def set_random_no_set(self, random_number_set):
        """
        Controls the random sampling
        Parameters:
        ----------
        random_number_set: int
            Used to control the set of pseudo random numbers used by
            the distributions in the simulation.
        """
        self.random_number_set = random_number_set
        self.init_sampling()

    def init_sampling(self):
        """
        Create the distributions used by the model and initialise
        the random seeds of each.
        """
        # produce n non-overlapping streams
        seed_sequence = np.random.SeedSequence(self.random_number_set)
        self.seeds = seed_sequence.spawn(self.n_streams)

        # create distributions

        # inter-arrival time distributions
        self.arrival_strokes = Exponential(
            self.iat_strokes, random_seed=self.seeds[0]
        )

        self.acute_los = Lognormal(
            self.acute_los_mean, self.acute_los_std, random_seed=self.seeds[1]
        )

    def init_results_variables(self):
        """
        Initialise all of the experiment variables used in results
        collection.  This method is called at the start of each run
        of the model
        """
        # variable used to store results of experiment
        self.results = {}
        self.results["n_arrivals"] = 0
        self.results["waiting_acute"] = []

## 🥵 Warm-up period

In [6]:
def warmup_complete(warm_up_period, env, args):
    """
    End of warm-up period event. Used to reset results collection variables.

    Parameters:
    ----------
    warm_up_period: float
        Duration of warm-up period in simultion time units

    env: simpy.Environment
        The simpy environment

    args: Experiment
        The simulation experiment that contains the results being collected.
    """
    yield env.timeout(warm_up_period)
    trace(f"{env.now:.2f}: 🥵 Warm up complete.")
    
    args.init_results_variables()

## 4. Pathway process logic

The key things to recognise are 

* We include a optional parameter called `collection_results` that defaults to `True`. We may set this `False` in our functions that setup initial conditions

In [7]:
def acute_stroke_pathway(patient_id, env, args):
    """Process a patient through the acute ward
    Simpy generator function.
    
    Parameters:
    -----------
    patient_id: int
        A unique id representing the patient in the process

    env: simpy.Environment
        The simulation environment

    args: Experiment
        Container class for the simulation parameters/results.
    """
    arrival_time = env.now

    with args.acute_ward.request() as acute_bed_request:
        yield acute_bed_request
        
        acute_admit_time = env.now
        wait_for_acute = acute_admit_time - arrival_time
               
        args.results['waiting_acute'].append(wait_for_acute)
        
        trace(f"{env.now:.2f}: Patient {patient_id} admitted to acute ward." \
              + f"(waited {wait_for_acute:.2f} days)")
        
        # Simulate acute care treatment
        acute_care_duration = args.acute_los.sample()
        yield env.timeout(acute_care_duration)
        
        trace(f"{env.now:.2f}: Patient {patient_id} discharged.")

## 4. Arrivals generator

This is a standard arrivals generator. We create stroke arrivals according to their distribution.

In [33]:
def stroke_arrivals_generator(env, args):
    """
    Arrival process for strokes.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_id in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_strokes.sample()
        yield env.timeout(inter_arrival_time)

        args.results["n_arrivals"] += 1
        
        trace(f"{env.now:.2f}: Stroke arrival.")

        # patient enters pathway
        env.process(acute_stroke_pathway(patient_id, env, args))

## 5. Single run function

In [34]:
def single_run(
    experiment, 
    rep=0,
    wu_period=WU_PERIOD,
    rc_period=RC_PERIOD
):
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/paramaters to use with model

    rep: int
        The replication number.

    wu_period: float, optional (default=WU_PERIOD)
        Warm-up period

    rc_period: float, optional (default=RC_PERIOD)
        The run length of the model
    """

    # reset all results variables to zero and empty
    experiment.init_results_variables()

    # set random number set to the replication no.
    # this controls sampling for the run.
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment()

    # simpy resources
    experiment.acute_ward = simpy.Resource(env, experiment.n_acute_beds)

    # schedule a warm_up period
    env.process(warmup_complete(wu_period, env, experiment))
    
    # we pass all arrival generators to simpy 
    env.process(stroke_arrivals_generator(env, experiment))

    # run model
    env.run(until=wu_period+rc_period)

    # quick stats
    results = {}
    results['mean_acute_wait'] = np.array(
        experiment.results["waiting_acute"]
    ).mean()

    # return single run results
    return results

## Quick check 1: No warm-up

In [35]:
TRACE = True
experiment = Experiment()
results = single_run(experiment, rep=0, wu_period=0.0, rc_period=5.0)
results

0.00: 🥵 Warm up complete.
3.29: Stroke arrival.
3.29: Patient 1 admitted to acute ward.(waited 0.00 days)
4.06: Stroke arrival.
4.06: Patient 2 admitted to acute ward.(waited 0.00 days)


{'mean_acute_wait': 0.0}

In [36]:
# check how many patient waiting times recorded.
experiment.results

{'n_arrivals': 2, 'waiting_acute': [0.0, 0.0]}

## Quick check 1: Include a warm-up

In [37]:
TRACE = True
experiment = Experiment()
results = single_run(experiment, rep=0, wu_period=5.0, rc_period=1.0)
results

3.29: Stroke arrival.
3.29: Patient 1 admitted to acute ward.(waited 0.00 days)
4.06: Stroke arrival.
4.06: Patient 2 admitted to acute ward.(waited 0.00 days)
5.00: 🥵 Warm up complete.
5.31: Stroke arrival.
5.31: Patient 3 admitted to acute ward.(waited 0.00 days)
5.53: Stroke arrival.
5.53: Patient 4 admitted to acute ward.(waited 0.00 days)
5.76: Stroke arrival.
5.76: Patient 5 admitted to acute ward.(waited 0.00 days)


{'mean_acute_wait': 0.0}

In [38]:
# check how many patient waiting times recorded.
experiment.results

{'n_arrivals': 3, 'waiting_acute': [0.0, 0.0, 0.0]}